In [56]:
import pgeocode

# Initialize the Nominatim object for Germany
nomi = pgeocode.Nominatim('DE')

# Query all postal codes for a specific city (e.g., "Berlin")
#city_data = nomi.query_location('Baden-Württemberg')  # Replace with a specific postal code
# print(city_data)
zip_data = nomi.query_postal_code('76669') 
#zip_data1 = nomi.query_location('Heidelberg',top_k=5) 
print(zip_data)
print(dir(nomi))

postal_code                   76669
country_code                     DE
place_name            Bad Schönborn
state_name        Baden-Württemberg
state_code                       BW
county_name        Karlsruhe Region
county_code                    82.0
community_name            Karlsruhe
community_code               8215.0
latitude                    49.2069
longitude                    8.6436
accuracy                        4.0
Name: 0, dtype: object
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data', '_data_frame', '_data_path', '_fuzzy_search', '_get_data', '_index_postal_codes', '_normalize_postal_code', '_str_contains_search', 'country', 'query_location', 'query_postal_

In [13]:
admin_level_country = 2
admin_level_bundesland = 4
admin_level_regierungsbezirk = 5
admin_level_landkreis = 6
admin_level_stadt = 8

In [146]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Berlin"
# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["name"~"^({name})$",i][boundary=administrative](area.searchArea);
out center;
"""

location_annotation = {
    "city": ["Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim"],  # admin level <= 8
}

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    print(f"Relation ID: {relation.id}")
    print(f"Latitude: {relation.center_lat}")
    print(f"Longitude: {relation.center_lon}")
    print("Tags:")
    for tag_key, tag_value in relation.tags.items():
        print(f"  {tag_key}: {tag_value}")
    print("\n")

Relation ID: 62422
Latitude: 52.5068768
Longitude: 13.4247530
Tags:
  ISO3166-2: DE-BE
  TMC:cid_58:tabcd_1:Class: Area
  TMC:cid_58:tabcd_1:LCLversion: 12.0
  TMC:cid_58:tabcd_1:LocationCode: 266
  admin_level: 4
  alt_name:vi: Béc-lin
  boundary: administrative
  coat_of_arms: File:Coat of arms of Berlin.svg
  contact:facebook: http://www.facebook.com/Berlin
  de:amtlicher_gemeindeschluessel: 11000000
  de:place: city
  de:regionalschluessel: 110000000000
  flag: File:Flag of Berlin.svg
  geographical_region: Barnim;Berliner Urstromtal;Teltow;Nauener Platte
  license_plate_code: B
  name: Berlin
  name:af: Berlyn
  name:als: Berlin
  name:am: በርሊን
  name:an: Berlín
  name:ang: Berlin
  name:ar: برلين
  name:arc: ܒܪܠܝܢ
  name:ast: Berlín
  name:az: Berlin
  name:ba: Берлин
  name:bar: Berlin
  name:bat-smg: Berlīns
  name:be: Берлін
  name:be-tarask: Бэрлін
  name:bg: Берлин
  name:bi: Berlin
  name:bn: বার্লিন
  name:bo: པེ་ལིང་
  name:br: Berlin
  name:bs: Berlin
  name:ca: Berlín
 

In [ ]:
import overpy
import re

# Initialize the Overpass API
api = overpy.Overpass()

# List of locations you want to query
locations = [
    "Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim",
    "Karlsruhe", "Deutschland", "Österreich", "Italien",
    "Rhein-Neckar-Kreis", "Baden-Württemberg", "Odenwaldkreis"
]

# Escape location names to handle any special regex characters
escaped_locations = [re.escape(location) for location in locations]

# Create a regex pattern for the location names
locations_pattern = '|'.join(escaped_locations)

# Construct the Overpass QL query with proper OR conditions
query = f"""
[out:json][timeout:60];
(
  relation["name"~"^({locations_pattern})$",i]["boundary"="administrative"];
  relation["name:de"~"^({locations_pattern})$",i]["boundary"="administrative"];
);
out center tags;
"""

try:
    # Run the query
    result = api.query(query)

    # Intermediate dictionary to track the highest admin_level for each location
    # Key: location name, Value: admin_level (integer)
    location_admin_levels = {}

    # Process each relation (location)
    for relation in result.relations:
        # Prefer 'name:de' over 'name' if available
        location_name = relation.tags.get("name:de") or relation.tags.get("name", "Unknown")
        location_name = location_name.strip()

        admin_level_str = relation.tags.get("admin_level")

        # Convert admin_level to integer if possible, else assign 0
        try:
            admin_level = int(admin_level_str) if admin_level_str is not None else 0
        except ValueError:
            admin_level = 0

        # Update the highest admin_level for the location
        if location_name in location_admin_levels:
            if admin_level < location_admin_levels[location_name]:
                location_admin_levels[location_name] = admin_level
        else:
            location_admin_levels[location_name] = admin_level

    # Initialize the final dictionary with admin_level as keys and list of locations as values
    admin_levels_dict = {}

    for location in locations:
        # Retrieve the admin_level for the location, default to 0 if not found
        admin_level = location_admin_levels.get(location, 0)

        # Initialize the list for this admin_level if not already present
        if admin_level not in admin_levels_dict:
            admin_levels_dict[admin_level] = []

        # Add the location to the corresponding admin_level
        admin_levels_dict[admin_level].append(location)

    # Print the dictionary in a structured format
    for admin_level in sorted(admin_levels_dict.keys()):
        print(f"Admin Level: {admin_level}")
        print(f"  Locations: {', '.join(admin_levels_dict[admin_level])}\n")

except overpy.exception.OverpassTooManyRequests:
    print("Error: Too many requests. Please try again later.")
except overpy.exception.OverpassGatewayTimeout:
    print("Error: The Overpass API timed out. Please try again later.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Admin Level: 2
  Locations: Deutschland, Österreich, Italien

Admin Level: 4
  Locations: Berlin, Baden-Württemberg

Admin Level: 6
  Locations: Heidelberg, Karlsruhe, Rhein-Neckar-Kreis, Odenwaldkreis

Admin Level: 8
  Locations: Wiesloch, Schwetzingen, Bensheim



In [147]:
print(admin_levels_dict)

{4: ['Berlin', 'Baden-Württemberg'], 6: ['Heidelberg', 'Karlsruhe', 'Rhein-Neckar-Kreis', 'Odenwaldkreis'], 8: ['Wiesloch', 'Schwetzingen', 'Bensheim'], 2: ['Deutschland', 'Österreich', 'Italien']}


In [119]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Heidelberg"
admin_level = 2
# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["note"~"{name}"][boundary=postal_code](area.searchArea);
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    print(f"Relation ID: {relation.id}")
    print(f"Latitude: {relation.center_lat}")
    print(f"Longitude: {relation.center_lon}")
    print("Tags:")
    for tag_key, tag_value in relation.tags.items():
        print(f"  {tag_key}: {tag_value}")
    print("\n")


Relation ID: 1186414
Latitude: 49.3938369
Longitude: 8.7306991
Tags:
  boundary: postal_code
  note: 69117 Heidelberg
  postal_code: 69117
  postal_code_level: 8
  ref: 69117
  type: boundary


Relation ID: 1186421
Latitude: 49.4179176
Longitude: 8.6194847
Tags:
  boundary: postal_code
  note: 69123 Heidelberg
  postal_code: 69123
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010
  type: boundary


Relation ID: 1186427
Latitude: 49.4230453
Longitude: 8.7558743
Tags:
  boundary: postal_code
  note: 69118 Heidelberg
  postal_code: 69118
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010
  type: boundary


Relation ID: 1186432
Latitude: 49.4197653
Longitude: 8.6908328
Tags:
  boundary: postal_code
  note: 69120 Heidelberg
  postal_code: 69120
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010


In [5]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Define the latitude, longitude, radius (in meters), and admin_level
latitude = 49.4058478 
longitude = 8.6835826 
radius = 10_000 
admin_level = 6  

# Overpass QL query to search around a point for relations within a radius
query = f"""
[out:json][timeout:60];
(
  relation["boundary"="administrative"]["admin_level"="{admin_level}"](around:{radius},{latitude},{longitude});
);
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    print(f"Relation ID: {relation.id}")
    print(f"Latitude: {relation.center_lat}")
    print(f"Longitude: {relation.center_lon}")
    print("Tags:")
    for tag_key, tag_value in relation.tags.items():
        print(f"  {tag_key}: {tag_value}")
    print("\n")


Relation ID: 62577
Latitude: 49.4000032
Longitude: 8.7758357
Tags:
  TMC:cid_58:tabcd_1:Class: Area
  TMC:cid_58:tabcd_1:LCLversion: 8.00
  TMC:cid_58:tabcd_1:LocationCode: 623
  admin_level: 6
  border_type: county
  boundary: administrative
  de:amtlicher_gemeindeschluessel: 08226
  de:place: county
  de:regionalschluessel: 08226
  license_plate_code: HD
  name: Rhein-Neckar-Kreis
  name:ar: منطقة راين نيكار
  name:pfl: Rhoi-Negger-Grais
  name:ru: Рейн-Неккар
  ref:nuts:3: DE128
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Kreisgrenzen_Deutschland_2005
  type: boundary
  wikidata: Q8177
  wikipedia: de:Rhein-Neckar-Kreis


Relation ID: 62691
Latitude: 49.5004160
Longitude: 8.5020752
Tags:
  TMC:cid_58:tabcd_1:Class: Area
  TMC:cid_58:tabcd_1:LCLversion: 8.00
  TMC:cid_58:tabcd_1:LocationCode: 526
  admin_level: 6
  boundary: administrative
  de:amtlicher_gemeindeschluessel: 08222000
  de:place: city
  de:regionalschluessel: 082220000000
  license_plate_code: MA
  na

In [69]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Define the latitude, longitude, radius (in meters), and admin_level
latitude = 49.4058478
longitude = 8.6835826
radius = 100

# Overpass QL query to search around a point for ways with the "highway" tag within a radius
query = f"""
[out:json][timeout:60];
way["highway"]["name"](around:{radius},{latitude},{longitude});
out body;
"""

# Run the query
result = api.query(query)

# Process ways
for way in result.ways:
    # print(f"Way ID: {way.id}")
    # print("Tags:")
    # for tag_key, tag_value in way.tags.items():
    #     print(f"  {tag_key}: {tag_value}")
    print(f"  Name: {way.tags['name']}")


  Name: Römerkreis
  Name: Ringstraße
  Name: Gaswerkstraße
  Name: Römerstraße
  Name: Römerkreis
  Name: Römerkreis
  Name: Römerkreis
  Name: Ringstraße
  Name: Kurfürsten-Anlage
  Name: Römerkreis
  Name: Römerstraße
  Name: Kurfürsten-Anlage
  Name: Ringstraße
  Name: Kurfürsten-Anlage
  Name: Römerstraße
  Name: Kurfürsten-Anlage
  Name: Kurfürsten-Anlage
  Name: Römerkreis
  Name: Römerstraße
  Name: Römerkreis
  Name: Alte Eppelheimer Straße
  Name: Römerkreis
  Name: Kurfürsten-Anlage


# Get Neighboring Countries

In [ ]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

name = "Deutschland"
admin_level = 2

# Overpass QL query to get neighboring countries of Germany
query = f"""
[out:json][timeout:60];
// Get the relation for the Location Area
relation["name"="{name}"]["boundary"="administrative"]["admin_level"="{admin_level}"]->.country;

// Get all ways that are members of this relation
way(r.country)->.country_ways;

// Find all relations that share these boundary ways
relation(bw.country_ways)["boundary"="administrative"]["type"="boundary"]["name:de"]["admin_level"="{admin_level}"]->.neighbors;

// Output the neighboring relations
relation.neighbors;
out center;
"""

# Run the query
result = api.query(query)

# Process relations
print(f"Neighboring Countries to {name}:")
for relation in result.relations:
    # print(f"Relation ID: {relation.id}")
    # print(f"Latitude: {relation.center_lat}")
    # print(f"Longitude: {relation.center_lon}")
    print(relation.tags.get('name:de'))
    # for tag_key, tag_value in relation.tags.items():
    #     print(f"  {tag_key}: {tag_value}")




Neighboring Countries to Germany:
Österreich
Polen
Dänemark
Deutschland
Tschechien
Schweiz
Belgien
Luxemburg
Frankreich
Niederlande


# Get Neighboring States

In [103]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Berlin"
admin_level = 4
# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
//Set search area to Germany
area["ISO3166-1"="DE"]->.searchArea;
// Get the relation for the Location Area
relation["name"~"{name}"]["boundary"="administrative"]["admin_level"="{admin_level}"](area.searchArea)->.bundesland;

// Get all ways that are members of this relation
way(r.bundesland)->.bundesland_ways;

// Find all relations that share these boundary ways
relation(bw.bundesland_ways)["boundary"="administrative"]["type"="boundary"]["name:de"]["admin_level"="{admin_level}"](area.searchArea)->.neighbors;

// Output the neighboring relations
relation.neighbors;
out center;
"""

# Run the query
result = api.query(query)

# Process relations
print(f"Neighboring Countries to {name}:")
for relation in result.relations:
    # print(f"Relation ID: {relation.id}")
    # print(f"Latitude: {relation.center_lat}")
    # print(f"Longitude: {relation.center_lon}")
    print(relation.tags.get('name:de'))
    # for tag_key, tag_value in relation.tags.items():
    #     print(f"  {tag_key}: {tag_value}")


Neighboring Countries to Berlin:
Berlin
Brandenburg


# Get Landkreise inside of State

In [33]:
import overpy
admin_level_country = 2
admin_level_bundesland = 4
admin_level_regierungsbezirk = 5
admin_level_landkreis = 6
admin_level_stadt = 8
# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Baden-Württemberg"

# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["name"~"{name}"]["boundary"="administrative"]["admin_level"="{admin_level_bundesland}"](area.searchArea)->.bundesland;

// convert relations object into an area object
.bundesland map_to_area -> .bundeslandArea;

// Find all relations that are a member of this area
relation["boundary"="administrative"]["admin_level"="{admin_level_landkreis}"](area.bundeslandArea)->.landkreise;

// Output the neighboring relations
relation.landkreise;
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    # print(f"Relation ID: {relation.id}")
    # print(f"Latitude: {relation.center_lat}")
    # print(f"Longitude: {relation.center_lon}")
    # print("Tags:")
    # for tag_key, tag_value in relation.tags.items():
    #     print(f"  {tag_key}: {tag_value}")
    # print("\n")
    print(f"  Name: {relation.tags['name']}")

  Name: Baden-Baden
  Name: Landkreis Rottweil
  Name: Landkreis Freudenstadt
  Name: Landkreis Rastatt
  Name: Landkreis Karlsruhe
  Name: Hohenlohekreis
  Name: Rems-Murr-Kreis
  Name: Pforzheim
  Name: Ulm
  Name: Karlsruhe
  Name: Landkreis Ludwigsburg
  Name: Main-Tauber-Kreis
  Name: Rhein-Neckar-Kreis
  Name: Landkreis Schwäbisch Hall
  Name: Landkreis Lörrach
  Name: Landkreis Calw
  Name: Landkreis Konstanz
  Name: Ortenaukreis
  Name: Neckar-Odenwald-Kreis
  Name: Landkreis Tuttlingen
  Name: Schwarzwald-Baar-Kreis
  Name: Mannheim
  Name: Ostalbkreis
  Name: Landkreis Waldshut
  Name: Landkreis Böblingen
  Name: Enzkreis
  Name: Landkreis Heilbronn
  Name: Heilbronn
  Name: Freiburg im Breisgau
  Name: Heidelberg
  Name: Landkreis Emmendingen
  Name: Landkreis Breisgau-Hochschwarzwald
  Name: Stuttgart
  Name: Landkreis Reutlingen
  Name: Landkreis Tübingen
  Name: Zollernalbkreis
  Name: Alb-Donau-Kreis
  Name: Landkreis Biberach
  Name: Landkreis Sigmaringen
  Name: Bodens

# Get City inside of Landkreis

In [ ]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Rhein-Neckar-Kreis"

# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["name"~"{name}"]["boundary"="administrative"]["admin_level"="{admin_level_landkreis}"](area.searchArea)->.landkreis;

// convert relations object into an area object
.landkreis map_to_area -> .landkreisArea;

// Find all relations that are a member of this area
relation["boundary"="administrative"]["admin_level"="{admin_level_stadt}"](area.landkreisArea)->.stadt;

// Output the neighboring relations
relation.stadt;
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    print(f"Relation ID: {relation.id}")
    # print(f"Latitude: {relation.center_lat}")
    # print(f"Longitude: {relation.center_lon}")
    # print("Tags:")
    # for tag_key, tag_value in relation.tags.items():
    #     print(f"  {tag_key}: {tag_value}")
    # print("\n")
    print(f"  Name: {relation.tags['name']}")

Relation ID: 453363
  Name: Eberbach
Relation ID: 453364
  Name: Schönbrunn
Relation ID: 453365
  Name: Reichartshausen
Relation ID: 453366
  Name: Lobbach
Relation ID: 453367
  Name: Spechbach
Relation ID: 453368
  Name: Epfenbach
Relation ID: 453369
  Name: Neidenstein
Relation ID: 453370
  Name: Waibstadt
Relation ID: 453371
  Name: Neckarbischofsheim
Relation ID: 453372
  Name: Sinsheim
Relation ID: 453373
  Name: Angelbachtal
Relation ID: 453374
  Name: Mühlhausen
Relation ID: 453375
  Name: Dielheim
Relation ID: 453376
  Name: Zuzenhausen
Relation ID: 453377
  Name: Eschelbronn
Relation ID: 453391
  Name: Neckargemünd
Relation ID: 453392
  Name: Wiesenbach
Relation ID: 453393
  Name: Bammental
Relation ID: 453394
  Name: Mauer
Relation ID: 453395
  Name: Meckesheim
Relation ID: 453396
  Name: Gaiberg
Relation ID: 453397
  Name: Leimen
Relation ID: 453398
  Name: Nußloch
Relation ID: 453399
  Name: Wiesloch
Relation ID: 453400
  Name: Rauenberg
Relation ID: 453401
  Name: Malsch
R

# Get Sreet inside of City

In [6]:
import overpy
admin_level_stadt = 8
# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Mühlhausen"

# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["name"="{name}"]["boundary"="administrative"]["admin_level"="{admin_level_stadt}"](area.searchArea)->.stadt; // relation(453374)->.stadt;

// convert relations object into an area object
.stadt map_to_area -> .stadtArea;

// Find all relations that are a member of this area
way["highway"]["name"](area.stadtArea)->.strasse;

// Output the neighboring relations
way.strasse;
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for ways in result.ways:
    # print(f"Relation ID: {ways.id}")
    # print(f"Latitude: {ways.center_lat}")
    # print(f"Longitude: {ways.center_lon}")
    # print("Tags:")
    # for tag_key, tag_value in ways.tags.items():
    #     print(f"  {tag_key}: {tag_value}")
    # print("\n")
    print(f"  Name: {ways.tags['name']}")

  Name: Bruchsaler Straße
  Name: Wappersdorfer Straße
  Name: Bahnhofstraße
  Name: An den Linden
  Name: Dorfstraße
  Name: Rocksdorfer Straße
  Name: Ziegelstraße
  Name: Kräheckerweg
  Name: Untere Mühlstraße
  Name: Hochstraße
  Name: Rotenberger Straße
  Name: Im Brühl
  Name: Odenwaldstraße
  Name: Am Klettenberg
  Name: Schulstraße
  Name: Hohlstraße
  Name: Bahnhofstraße
  Name: Hinterer Berg
  Name: Badgasse
  Name: Marktplatz
  Name: Schwabengasse
  Name: Schmiedgasse
  Name: Badberg
  Name: Schwarzer Winkel
  Name: Ziegelanger
  Name: Marktplatz
  Name: Schloßberg
  Name: Weingasse
  Name: Pointweg
  Name: Engelgasse
  Name: Lange Gasse
  Name: Vorderer Berg
  Name: Ziegelhütte
  Name: Malscher Straße
  Name: Malscher Straße
  Name: Mühlweiher
  Name: Wappersdorfer Straße
  Name: Schweppermannstraße
  Name: Föhrenweg
  Name: Erlenweg
  Name: Wappersdorfer Straße
  Name: Sulzau
  Name: Sulzau
  Name: Hofau
  Name: Blumenstraße
  Name: Gartenstraße
  Name: Am Alten Bahnhof
  

# Is relation contained in another relation? (clustering operation)

In [49]:
admin_level_country = 2
admin_level_bundesland = 4
admin_level_regierungsbezirk = 5
admin_level_landkreis = 6
admin_level_verwaltungsgemeinschaft = 7
admin_level_stadt = 8
admin_level_stadtteil = 9

import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Names of the district and city
district_name = "Neuenheim"
city_name = "Heidelberg"

# Overpass QL query to check if the district is within the city
query = f"""
[out:json][timeout:60];

// Get the area ID of the city
area[name="{city_name}"]["boundary"="administrative"]->.cityArea;

// Now get the relation of the district within that area
relation["name"="{district_name}"]["boundary"="administrative"](area.cityArea);

// Outputting only ids is an efficient way to determine if a match exists
out ids;
"""

# Run the query
result = api.query(query)

# Check if the district is within the city
if result.relations:
    print(f"The district '{district_name}' is within the city '{city_name}'.")
else:
    print(f"The district '{district_name}' is not within the city '{city_name}'.")


The district 'Neuenheim' is within the city 'Heidelberg'.


# Determine Bundesland of location

In [39]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# The place name you want to search for
place_name = "Bruchsal"

# First, fetch the place and get its center point
query_place = f"""
[out:json][timeout:25];
area["ISO3166-1"="DE"]->.searchArea;
// Fetch the relation for the place and get its center point
relation["name"="{place_name}"]["boundary"="administrative"](area.searchArea);
out center;
"""

# Run the query to get the place's center point
result_place = api.query(query_place)

if result_place.relations:
    place_relation = result_place.relations[0]
    center_lat = place_relation.center_lat
    center_lon = place_relation.center_lon

    # Now, use the center point to find the Bundesland containing it
    query_bundesland = f"""
    [out:json][timeout:25];

    // Use the center point to find containing administrative boundaries
    is_in({center_lat},{center_lon});
    // Select relations that are administrative boundaries at admin_level=4 (Bundesländer)
    relation(pivot)["boundary"="administrative"]["admin_level"="4"];
    out tags;
    """

    # Run the query to find the Bundesland
    result_bundesland = api.query(query_bundesland)

    if result_bundesland.relations:
        # Assuming there's only one Bundesland containing the point
        bundesland_relation = result_bundesland.relations[0]
        state_name = bundesland_relation.tags.get("name")
        print(f"The place '{place_name}' is in the Bundesland: {state_name}")
    else:
        print(f"The Bundesland for '{place_name}' could not be found.")
else:
    print(f"The place '{place_name}' could not be found.")




The place 'Bruchsal' is in the Bundesland: Baden-Württemberg


# Cluster locations into Hierarchy

In [ ]:
location_annotation = {
    "country": ["Deutschland", "Österreich", "Italien"],                                                # admin level = 2 / Land
    "state": ["Rhein-Neckar-Kreis", "Baden-Württemberg", "Odenwaldkreis"],                              # admin level 3 < x < 7 / 4:Bundesland, 5:Regierungsbezirk, 6:Landkreis
    "city": ["Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim"],                           # admin level = <= 8 / 8:Stadt, 9:Stadtteil, 10:Ortsteil 
    "zip": ["10178", "69115", "69168", "68723", "64625"],                                               # admin level = <= 8 / 8:Stadt, 9:Stadtteil, 10:Ortsteil 
    "street": ["Spandauer Str. 3", "Bergstraße 3", "Carl-Theodor-Straße 13", "Fabrikstraße 10"],
}

['Deutschland', 'Österreich', 'Italien']


In [ ]:
location_annotation = {
    "country": ["Deutschland", "Österreich", "Italien"],                                                # admin level = 2 / Land
    "state": ["Rhein-Neckar-Kreis", "Baden-Württemberg", "Odenwaldkreis"],                              # admin level 3 < x < 7 / 4:Bundesland, 5:Regierungsbezirk, 6:Landkreis
    "city": ["Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim"],                           # admin level = <= 8 / 8:Stadt, 9:Stadtteil, 10:Ortsteil 
    "street": ["Spandauer Str. 3", "Bergstraße 3", "Carl-Theodor-Straße 13", "Fabrikstraße 10"],
}

In [82]:
admin_level_country = 2
admin_level_bundesland = 4
admin_level_regierungsbezirk = 5
admin_level_landkreis = 6
admin_level_stadt = 8



def is_child(parent_name, parent_admin_level, child_name, child_admin_level, api):
    """
    This function uses the Overpass API to check whether a given child region is contained
    within the boundaries of a specified parent region. 

    Parameters
    ----------
    parent_name : str
        The name of the parent administrative region.
    parent_admin_level : int
        The administrative level of the parent region (e.g., 4 for states in Germany).
    child_name : str
        The name of the child administrative region.
    child_admin_level : Union[int, List[int]]
        Either a single administrative level (int) or a range as a list of two integers [lower, upper].
    api : overpy.Overpass
        An instance of the Overpass API for querying OpenStreetMap data.

    Returns
    -------
    bool
        True if the child administrative region is within the parent region; False otherwise.
    """
    try:
        # Handle single value or range for child_admin_level
        if isinstance(child_admin_level, int):
            admin_level_filter = f'["admin_level"="{child_admin_level}"]'
        elif isinstance(child_admin_level, list) and len(child_admin_level) == 2:
            admin_level_filter = f'["admin_level"~"^[{child_admin_level[0]}-{child_admin_level[1]}]$"]'
        else:
            raise ValueError("child_admin_level must be an int or a list of two integers.")

        # Overpass QL query
        query = f"""
        [out:json];
        area["ISO3166-1"="DE"]->.searchArea;
        relation["name"~"{parent_name}"]["boundary"="administrative"]["admin_level"="{parent_admin_level}"](area.searchArea)->.parent;

        // convert relations object into an area object
        .parent map_to_area -> .parentArea;

        // Find all relations that are a member of this area
        relation["name"~"{child_name}"]["boundary"="administrative"]{admin_level_filter}(area.parentArea)->.child;

        // Output the neighboring relations
        relation.child;
        out ids;
        """

        # Run the query
        result = api.query(query)
        
        # Return True if any child relations were found
        return bool(result.relations)

    except overpy.exception.OverpassBadRequest as e:
        logging.error(f"Bad Overpass Query: {e}")
        return False
    except ValueError as e:
        logging.error(f"Invalid Input: {e}")
        return False
    except Exception as e:
        logging.error(f"Unexpected Error: {e}")
        return False

    
import logging   
import overpy
# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
parent_name = "Baden-Württemberg"
parent_admin_level = 4
child_name = "Heidelberg"
child_admin_level_range = [6,8] # list or int
  
print(is_child(parent_name, parent_admin_level, child_name, child_admin_level_range, api))

True


# Get City name for zip

In [ ]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Specify the postal code for the query
postal_code = "76669"  # Replace with the desired postal code

# Overpass QL query to retrieve coordinates for the given postal code
query = f"""
[out:json][timeout:60];
area["boundary"="postal_code"]["postal_code"="{postal_code}"];
out center;
"""


result = api.query(query)

# Process areas
for area in result.areas:
    print(f"Note: {area.tags['note']}")

# Get surrogate for zip 

In [12]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
name = "Heidelberg"
admin_level = 2
# Overpass QL query to get basic information about a city
query = f"""
[out:json][timeout:60];
area["ISO3166-1"="DE"]->.searchArea;
relation["note"~"{name}"][boundary=postal_code](area.searchArea);
out center;
"""

# Run the query
result = api.query(query)

# Process relations
for relation in result.relations:
    print(f"Relation ID: {relation.id}")
    print(f"Latitude: {relation.center_lat}")
    print(f"Longitude: {relation.center_lon}")
    print("Tags:")
    for tag_key, tag_value in relation.tags.items():
        print(f"  {tag_key}: {tag_value}")
    print("\n")

Relation ID: 1186414
Latitude: 49.3938369
Longitude: 8.7306991
Tags:
  boundary: postal_code
  note: 69117 Heidelberg
  postal_code: 69117
  postal_code_level: 8
  ref: 69117
  type: boundary


Relation ID: 1186421
Latitude: 49.4179176
Longitude: 8.6194847
Tags:
  boundary: postal_code
  note: 69123 Heidelberg
  postal_code: 69123
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010
  type: boundary


Relation ID: 1186427
Latitude: 49.4230453
Longitude: 8.7558743
Tags:
  boundary: postal_code
  note: 69118 Heidelberg
  postal_code: 69118
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010
  type: boundary


Relation ID: 1186432
Latitude: 49.4197653
Longitude: 8.6908328
Tags:
  boundary: postal_code
  note: 69120 Heidelberg
  postal_code: 69120
  postal_code_level: 8
  source: http://wiki.openstreetmap.org/wiki/Import/Catalogue/Postleitzahlen_Deutschland_2010


In [ ]:
admin_level_country = 2
admin_level_bundesland = 4
admin_level_regierungsbezirk = 5
admin_level_landkreis = 6
admin_level_stadt = 8

location_annotation = {
    "country": ["Deutschland", "Österreich", "Italien"],  # admin level = 2 / Land
    "state": ["Rhein-Neckar-Kreis", "Baden-Württemberg", "Odenwaldkreis"],  # admin level 4
    "city": ["Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim"],  # admin level <= 8
}

def is_child(parent_id, child_name, child_admin_level, api):
    """
    This function uses the Overpass API to check whether a given child region is contained
    within the boundaries of a specified parent region. 

    Parameters
    ----------
    parent_name : str
        The name of the parent administrative region.
    parent_admin_level : int
        The administrative level of the parent region (e.g., 4 for states in Germany).
    child_name : str
        The name of the child administrative region.
    child_admin_level : Union[int, List[int]]
        Either a single administrative level (int) or a range as a list of two integers [lower, upper].
    api : overpy.Overpass
        An instance of the Overpass API for querying OpenStreetMap data.

    Returns
    -------
    bool
        True if the child administrative region is within the parent region; False otherwise.
    """
    try:
        # Handle single value or range for child_admin_level
        if isinstance(child_admin_level, int):
            admin_level_filter = f'["admin_level"="{child_admin_level}"]'
        elif isinstance(child_admin_level, list) and len(child_admin_level) == 2:
            admin_level_filter = f'["admin_level"~"^[{child_admin_level[0]}-{child_admin_level[1]}]$"]'
        else:
            raise ValueError("child_admin_level must be an int or a list of two integers.")

        # Overpass QL query
        query = f"""
        [out:json];
        //Use the ID of the parent to specify Search Area
        relation({parent_id})->.parent;

        // convert relations object into an area object
        .parent map_to_area -> .parentArea;

        // Find all relations that are a member of this area
        relation["name"~"{child_name}"]["boundary"="administrative"]{admin_level_filter}(area.parentArea)->.child;

        // Output the neighboring relations
        relation.child;
        out ids;
        """

        # Run the query
        result = api.query(query)
        
        # Return True if any child relations were found
        return result.relations[0].id if result.relations else None

    except overpy.exception.OverpassBadRequest as e:
        logging.error(f"Bad Overpass Query: {e}")
        return False
    except ValueError as e:
        logging.error(f"Invalid Input: {e}")
        return False
    except Exception as e:
        logging.error(f"Unexpected Error: {e}")
        return False

    
import logging   
import overpy
# Initialize the Overpass API
api = overpy.Overpass()

# Replace "Heidelberg" with the desired city name
parent_id = 285864
child_name = "Heidelberg"
child_admin_level_range = [6,8] # list or int

print(is_child(parent_id, child_name, child_admin_level_range, api))

285864


# Determine Proper Location Admin Levels

In [ ]:
import overpy
import re
import logging
from anytree import Node, RenderTree

api = overpy.Overpass()
admin_level_city = 8

locations = [
    "Berlin", "Heidelberg", "Wiesloch", "Schwetzingen", "Bensheim",
    "Karlsruhe", "Deutschland", "Österreich", "Italien",
    "Rhein-Neckar-Kreis", "Baden-Württemberg", "Odenwaldkreis", "Mühlhausen"
]

street_locations = [
    "Spandauer Straße", "Bergstraße", "Carl-Theodor-Straße", "Fabrikstraße"
]

def fetch_location_info(location_list):
    """
    Query Overpass for the given locations and return their best admin_level match.

    Parameters
    ----------
    location_list : list of str
        Location names to query.

    Returns
    -------
    dict
        {location_name: {'admin_level': int, 'id': int or None}}
    """
    escaped = [re.escape(loc) for loc in location_list]
    pattern = '|'.join(escaped)

    query = f"""
    [out:json][timeout:60];
    (
      relation["name"~"^({pattern})$",i]["boundary"="administrative"];
      relation["name:de"~"^({pattern})$",i]["boundary"="administrative"];
    );
    out center tags;
    """
    result = api.query(query)

    info = {}
    for r in result.relations:
        name = (r.tags.get("name:de") or r.tags.get("name", "Unknown")).strip()
        lvl_str = r.tags.get("admin_level", "")
        level = int(lvl_str) if lvl_str.isdigit() else 0
        if name not in info or level < info[name]['admin_level']:
            info[name] = {'admin_level': level, 'id': r.id}
    return info

def group_locations_by_admin_level(location_list, locations_data):
    """
    Group locations by their admin_level.

    Parameters
    ----------
    location_list : list of str
        The queried locations.
    locations_data : dict
        {location_name: {'admin_level': int, 'id': int or None}}

    Returns
    -------
    dict
        {admin_level: [{'name': str, 'id': int or None}, ...]}
    """
    grouped = {}
    for loc in location_list:
        data = locations_data.get(loc, {'admin_level': 0, 'id': None})
        level = data['admin_level']
        grouped.setdefault(level, []).append({'name': loc, 'id': data['id']})
    return grouped

def is_child(parent_id, child_name, api):
    """
    Check if child_name is within parent_id's area.

    Parameters
    ----------
    parent_id : int
        OSM relation ID of the parent area.
    child_name : str
        Name of the child area.
    api : overpy.Overpass
        Overpass API instance.

    Returns
    -------
    int or None
        Child's relation ID if found, else None.
    """
    query = f"""
    [out:json];
    relation({parent_id})->.parent;
    .parent map_to_area -> .parentArea;
    relation["name"~"^{re.escape(child_name)}$",i]["boundary"="administrative"](area.parentArea);
    out ids;
    """
    try:
        result = api.query(query)
        return result.relations[0].id if result.relations else None
    except (overpy.exception.OverpassBadRequest, Exception):
        return None

def build_hierarchy(locations_by_level):
    """
    Build a hierarchy of administrative nodes and add one street for nodes >= admin_level 8.

    Parameters
    ----------
    locations_by_level : dict
        {admin_level: [{'name': str, 'id': int or None}, ...]}

    Returns
    -------
    dict
        {node_id: anytree.Node} representing the hierarchy.
    """
    hierarchy_nodes = {}

    def find_one_street_in_area(parent_id, street_name):
        """
        Find one street within an area.

        Parameters
        ----------
        parent_id : int
            Parent area's relation ID.
        street_name : str
            Street name to find.

        Returns
        -------
        tuple or None
            (street_name, way_id) if found, else None.
        """
        query = f"""
        [out:json];
        relation({parent_id})->.parent;
        .parent map_to_area -> .parentArea;
        way["highway"]["name"~"^{re.escape(street_name)}$"](area.parentArea);
        out ids;
        """
        try:
            res = api.query(query)
            if res.ways:
                # Just return the first found way and ignore others
                return (street_name, res.ways[0].id)
        except:
            pass
        return None

    def create_or_get_node(name, node_id, level):
        """
        Create or retrieve a node and link it to its parent. Add one street if level >= 8.

        Parameters
        ----------
        name : str
            Location name.
        node_id : int or None
            Location's relation ID.
        level : int
            Location's admin_level.

        Returns
        -------
        anytree.Node
            The node representing this location.
        """
        if node_id in hierarchy_nodes:
            return hierarchy_nodes[node_id]

        node = Node(name, id=node_id)
        hierarchy_nodes[node_id] = node

        if level == 2:
            return node

        # If admin_level >= 8, look for one street in this area and add it as child
        if node_id and level >= admin_level_city:
            for street in street_locations:
                street_info = find_one_street_in_area(node_id, street)
                if street_info:
                    s_name, s_id = street_info
                    Node(s_name, id=s_id, parent=node)

        # Try to find a parent for this node
        higher_levels = sorted([l for l in locations_by_level if l < level], reverse=True)
        for hl in higher_levels:
            for p_info in locations_by_level[hl]:
                p_id = p_info['id']
                if p_id and is_child(p_id, name, api):
                    p_node = create_or_get_node(p_info['name'], p_id, hl)
                    node.parent = p_node
                    return node

        return node

    for level in sorted(locations_by_level.keys(), reverse=True):
        for entry in locations_by_level[level]:
            create_or_get_node(entry['name'], entry['id'], level)

    return hierarchy_nodes

try:
    locations_data = fetch_location_info(locations)
    locations_by_level = group_locations_by_admin_level(locations, locations_data)
    hierarchy_nodes = build_hierarchy(locations_by_level)

    # Print hierarchy
    root_nodes = [n for n in hierarchy_nodes.values() if n.is_root]
    for root in root_nodes:
        for pre, _, node in RenderTree(root):
            streets_info = f" (Streets: {', '.join(node.streets)})" if hasattr(node, 'streets') else ""
            print(f"{pre}{node.name} (ID: {node.id}){streets_info}")

except overpy.exception.OverpassTooManyRequests:
    print("Error: Too many requests. Please try again later.")
except overpy.exception.OverpassGatewayTimeout:
    print("Error: The Overpass API timed out. Please try again later.")
except Exception as e:
    print(f"Unexpected error: {e}")


Deutschland (ID: 51477)
├── Baden-Württemberg (ID: 62611)
│   ├── Rhein-Neckar-Kreis (ID: 62577)
│   │   ├── Wiesloch (ID: 453399)
│   │   │   └── Bergstraße (ID: 5071422)
│   │   ├── Schwetzingen (ID: 454165)
│   │   │   └── Carl-Theodor-Straße (ID: 10701748)
│   │   └── Mühlhausen (ID: 437377)
│   ├── Heidelberg (ID: 285864)
│   └── Karlsruhe (ID: 62518)
├── Bensheim (ID: 537055)
│   └── Fabrikstraße (ID: 5976272)
├── Odenwaldkreis (ID: 62596)
└── Berlin (ID: 62422)
Österreich (ID: 16239)
Italien (ID: 365331)


Admin Level: 2
  Locations: Deutschland, Österreich, Italien

Admin Level: 4
  Locations: Berlin, Baden-Württemberg

Admin Level: 6
  Locations: Heidelberg, Karlsruhe, Rhein-Neckar-Kreis, Odenwaldkreis

Admin Level: 8
  Locations: Wiesloch, Schwetzingen, Bensheim

Starting to build hierarchy...


Processing locations at admin level 8...
Wiesloch 453399 8
Processing 'Wiesloch' at admin level 8 (ID: 453399)...
  Checking for parents at admin level 6...
    Checking if 'Wiesloch' is a child of 'Heidelberg' (ID: 285864)...
    Checking if 'Wiesloch' is a child of 'Karlsruhe' (ID: 62518)...
    Checking if 'Wiesloch' is a child of 'Rhein-Neckar-Kreis' (ID: 62577)...
Processing 'Rhein-Neckar-Kreis' at admin level 6 (ID: 62577)...
  Checking for parents at admin level 4...
    Checking if 'Rhein-Neckar-Kreis' is a child of 'Berlin' (ID: 62422)...
    Checking if 'Rhein-Neckar-Kreis' is a child of 'Baden-Württemberg' (ID: 62611)...
Processing 'Baden-Württemberg' at admin level 4 (ID: 62611)...
  Checking for parents at admin level 2...
    Checking if 'Baden-Württemberg' is a child of 'Deutschland' (ID: 51477)...
Processing 'Deutschland' at admin level 2 (ID: 51477)...
  'Deutschland' is at admin_level 2, treating it as a root node. No parent search needed.
    Added 'Baden-Württemberg' as child of 'Deutschland'
    Added 'Rhein-Neckar-Kreis' as child of 'Baden-Württemberg'
    Added 'Wiesloch' as child of 'Rhein-Neckar-Kreis'
Schwetzingen 454165 8
Processing 'Schwetzingen' at admin level 8 (ID: 454165)...
  Checking for parents at admin level 6...
    Checking if 'Schwetzingen' is a child of 'Heidelberg' (ID: 285864)...
    Checking if 'Schwetzingen' is a child of 'Karlsruhe' (ID: 62518)...
    Checking if 'Schwetzingen' is a child of 'Rhein-Neckar-Kreis' (ID: 62577)...
    Added 'Schwetzingen' as child of 'Rhein-Neckar-Kreis'
Bensheim 537055 8
Processing 'Bensheim' at admin level 8 (ID: 537055)...
  Checking for parents at admin level 6...
    Checking if 'Bensheim' is a child of 'Heidelberg' (ID: 285864)...
    Checking if 'Bensheim' is a child of 'Karlsruhe' (ID: 62518)...
    Checking if 'Bensheim' is a child of 'Rhein-Neckar-Kreis' (ID: 62577)...
    Checking if 'Bensheim' is a child of 'Odenwaldkreis' (ID: 62596)...
  Checking for parents at admin level 4...
    Checking if 'Bensheim' is a child of 'Berlin' (ID: 62422)...
    Checking if 'Bensheim' is a child of 'Baden-Württemberg' (ID: 62611)...
  Checking for parents at admin level 2...
    Checking if 'Bensheim' is a child of 'Deutschland' (ID: 51477)...
    Added 'Bensheim' as child of 'Deutschland'

Processing locations at admin level 6...
Heidelberg 285864 6
Processing 'Heidelberg' at admin level 6 (ID: 285864)...
  Checking for parents at admin level 4...
    Checking if 'Heidelberg' is a child of 'Berlin' (ID: 62422)...
    Checking if 'Heidelberg' is a child of 'Baden-Württemberg' (ID: 62611)...
    Added 'Heidelberg' as child of 'Baden-Württemberg'
Karlsruhe 62518 6
Processing 'Karlsruhe' at admin level 6 (ID: 62518)...
  Checking for parents at admin level 4...
    Checking if 'Karlsruhe' is a child of 'Berlin' (ID: 62422)...
    Checking if 'Karlsruhe' is a child of 'Baden-Württemberg' (ID: 62611)...
    Added 'Karlsruhe' as child of 'Baden-Württemberg'
Rhein-Neckar-Kreis 62577 6
Odenwaldkreis 62596 6
Processing 'Odenwaldkreis' at admin level 6 (ID: 62596)...
  Checking for parents at admin level 4...
    Checking if 'Odenwaldkreis' is a child of 'Berlin' (ID: 62422)...
    Checking if 'Odenwaldkreis' is a child of 'Baden-Württemberg' (ID: 62611)...
  Checking for parents at admin level 2...
    Checking if 'Odenwaldkreis' is a child of 'Deutschland' (ID: 51477)...
    Added 'Odenwaldkreis' as child of 'Deutschland'

Processing locations at admin level 4...
Berlin 62422 4
Processing 'Berlin' at admin level 4 (ID: 62422)...
  Checking for parents at admin level 2...
    Checking if 'Berlin' is a child of 'Deutschland' (ID: 51477)...
    Added 'Berlin' as child of 'Deutschland'
Baden-Württemberg 62611 4

Processing locations at admin level 2...
Deutschland 51477 2
Österreich 16239 2
Processing 'Österreich' at admin level 2 (ID: 16239)...
  'Österreich' is at admin_level 2, treating it as a root node. No parent search needed.
Italien 365331 2
Processing 'Italien' at admin level 2 (ID: 365331)...
  'Italien' is at admin_level 2, treating it as a root node. No parent search needed.

Hierarchy building complete.
Deutschland (ID: 51477)
├── Baden-Württemberg (ID: 62611)
│   ├── Rhein-Neckar-Kreis (ID: 62577)
│   │   ├── Wiesloch (ID: 453399)
│   │   └── Schwetzingen (ID: 454165)
│   ├── Heidelberg (ID: 285864)
│   └── Karlsruhe (ID: 62518)
├── Bensheim (ID: 537055)
├── Odenwaldkreis (ID: 62596)
└── Berlin (ID: 62422)
Österreich (ID: 16239)
Italien (ID: 365331)